In [1]:
#导入相关包
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import EarlyStopping
#创建模型 -待完成
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN,LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
import time
import os

In [2]:
#函数预处理模块

#读取文件路径为入参的csv文件。返回DataFram类型的 文件内容
def mLoad_Data_Fun(mFileNamePara_str:str)->pd.DataFrame:
    mOrgData_pd = pd.read_csv(mFileNamePara_str)#'D:\\mMLP\\StockData\\mdata0.csv')
    return mOrgData_pd


#数据预处理   高、开、低、收,成交量 的数据有意义的是其相对于昨日的涨跌幅
def mPreDeal_OneData_Fun(mParaPrice_array:np.array)->np.array:
    mDataTemp_array = []
    i = 1
    while(i<len(mParaPrice_array)):
        #因为最大涨跌幅是10%，所以*10 将数据限制在-1-1间
        mData_D = (mParaPrice_array[i] / mParaPrice_array[i-1] - 1) * 10
        mDataTemp_array.append(mData_D)
        i += 1
    mDataTemp_array = np.array(mDataTemp_array)
    return mDataTemp_array


#数据预处理  高-收、收-开、开-低和昨日收盘价格 的比率=
def mUpDownShiti_Percent_Fun(mHighPara_array:np.array,mOpenPara_array:np.array,mLowPara_array:np.array,\
                             mClosePara_array:np.array,):
    mUpPercentTemp_np = []
    mDownPercentTemp_np = []
    mShitiPercentTemp_np = []
    
    i = 1
    while(i < len(mClosePara_array)):
        
        #因为 这3个值的最大值为 20%(涨跌幅10%，天地板时)，所以* 5 将其限制在 0-1之间
        if mOpenPara_array[i] > mClosePara_array[i]:
            #上影线
            mUpPercentTemp_d = (mHighPara_array[i] - mOpenPara_array[i]) / mClosePara_array[i-1] * 5
            #下影线
            mDownPercentTemp_d = (mClosePara_array[i] - mLowPara_array[i]) / mClosePara_array[i-1] * 5
        else:
            #上影线
            mUpPercentTemp_d = (mHighPara_array[i] - mClosePara_array[i]) / mClosePara_array[i-1] * 5
            #下影线
            mDownPercentTemp_d = (mOpenPara_array[i] - mLowPara_array[i]) / mClosePara_array[i-1] * 5
            
        #实体 因为 开、收的大小关系 本身也含有信息，所以不取其绝对值
        mShitiPercentTemp_d = (mOpenPara_array[i] - mClosePara_array[i]) / mClosePara_array[i-1] * 5
            
        mUpPercentTemp_np.append(mUpPercentTemp_d)
        mDownPercentTemp_np.append(mDownPercentTemp_d)
        mShitiPercentTemp_np.append(mShitiPercentTemp_d)
        
        i += 1
        
    mUpPercentTemp_np = np.array(mUpPercentTemp_np)
    mDownPercentTemp_np = np.array(mDownPercentTemp_np)
    mShitiPercentTemp_np = np.array(mShitiPercentTemp_np)
    return mUpPercentTemp_np,mDownPercentTemp_np,mShitiPercentTemp_np



#判断类别y的预处理 
def mPreDeal_Category_Y_Fun(mClosePara_array:np.array)->np.array:
    """
    按 往后5天内，close的涨跌幅划分区间
    0：跌幅大于3个点的 
    1：跌幅在 1.5 - 0 之间的
    2：涨幅在 0 - 1.5之间的
    3：涨幅在 1.5 -3 之间的
    4： 涨幅大于 3个点的
    
    """
    mReturn_array = []
    i = 1
    while(i<len(mClosePara_array)):
        #如果后续天数不足5天，则取后续剩余天数为判断天数
        if(i+4 < len(mClosePara_array)):
            mCloseTemp_np = mClosePara_array[i:i+5]
        else:
            mCloseTemp_np = mClosePara_array[i:]
        
        mMaxClose_f = np.max(mCloseTemp_np)
        #将张地方转为百分点新式，即3 表示张 3个百分点
        mRate = ( mMaxClose_f / mClosePara_array[i-1]  -1 ) * 100 
        
        if mRate > 3:
            mReturn_array.append(4)
        elif mRate > 1.5:
            mReturn_array.append(3)
        elif mRate > 0:
            mReturn_array.append(2)
        elif mRate > -1.5:
            mReturn_array.append(1)
        else:
            mReturn_array.append(0)
        
        i += 1
    
    mReturn_array = np.array(mReturn_array)
    
    return mReturn_array

#数据总预处理-将数据处理成
def mPreDeal_Total_Fun(mData_pd:pd.DataFrame):
    mCloseTemp_array = np.array(mData_pd.loc[:,'close'])
    mOpenTemp_array = np.array(mData_pd.loc[:,'open'])
    mHighTemp_array = np.array(mData_pd.loc[:,'high'])
    mLowTemp_array = np.array(mData_pd.loc[:,'low'])
    mVolumeTemp_array = np.array(mData_pd.loc[:,'volume'])
    
    #忘了RSI指标的意义，暂时先搁置
    #mRSI_array = np.array(mData_pd.loc[1:,'RSI'])
    
    #对高、开、低、收、成交量 进行预处理，获取其相对于其昨日数据的比率
    mPreCloseSimple_array = mPreDeal_OneData_Fun(mCloseTemp_array)
    mPreOpenSimple_array = mPreDeal_OneData_Fun(mOpenTemp_array)
    mPreHighSimple_array = mPreDeal_OneData_Fun(mHighTemp_array)
    mPreLowSimple_array = mPreDeal_OneData_Fun(mLowTemp_array)
    mPreVolume_array = mPreDeal_OneData_Fun(mVolumeTemp_array)
    
    #获取上下影线、实体相对于昨日收盘价的比率
    mUpPercent_np,mDownPercent_np,mShitiPercent_np = mUpDownShiti_Percent_Fun(mHighTemp_array,mOpenTemp_array,mLowTemp_array,\
                                                                              mCloseTemp_array)
  
    
    mPreFinalY_list = mPreDeal_Category_Y_Fun(mCloseTemp_array)
    
    #7:mRSI_array,
    mPreFinalData_pd = {0:mPreCloseSimple_array, 1:mPreOpenSimple_array, 2:mPreHighSimple_array , 3:mPreLowSimple_array \
                    , 4:mPreVolume_array ,5:mUpPercent_np,6:mDownPercent_np,7:mShitiPercent_np,'y':mPreFinalY_list}
    mPreFinalData_pd = pd.DataFrame(mPreFinalData_pd)
    
    return mPreFinalData_pd


#数据预处理 -将mPreDeal_Total_Fun预处理后的数据整理成RNN模型的输入类型
def mPre_Change_RNNPara_Fun(mPreFinalPara_pd:pd.DataFrame, mStepPara_I=10):
    #
    x = []
    y = []
    mRangeLen_I = len(mPreFinalPara_pd)-mStepPara_I 
    for i in range(mRangeLen_I):
        mDataTemp = np.array(mPreFinalPara_pd.loc[i:i+mStepPara_I-1,:].drop('y',axis=1)) 
        x.append(mDataTemp) 
        y.append(mPreFinalPara_pd.loc[i+mStepPara_I-1,'y']) 
        

    x = np.array(x)
    y = pd.Series(y)
    
    return x,y


#转化为one-hot类型
def mYChange_One_hot_Fun(mYTrainPara_list:pd.Series):
    y_train = np_utils.to_categorical(mYTrainPara_list, 5)
    return y_train

In [5]:
#""" 单支股票进行训练
mOrgData_df = mLoad_Data_Fun('E:\\mMachineLerning\\Own\\RNN\\mTrain_StockData\\000012.csv')
mPreData_df = mPreDeal_Total_Fun(mOrgData_df)
mPreData_df.head()
#"""

,0,1,2,3,4,5,6,7,y
0,-0.301339,0.510264,-0.237838,-0.489387,-2.692994,0.039062,0.348772,0.150670,0
1,-0.684695,-0.418527,-0.393134,0.037198,-3.253033,0.051784,0.000000,0.281933,0
2,-1.000618,-0.908561,-0.783862,-1.000618,-0.220508,0.117356,0.000000,0.321186,0
3,0.123542,-0.666240,-0.625391,-0.974605,-0.155220,0.082361,0.487303,-0.061771,3
4,0.162712,0.157859,0.173449,1.034221,-1.037201,0.088136,0.098305,-0.064407,3


In [6]:
#"""单支股票进行训练
mPreDataRnn_x,mPreDataRnn_y = mPre_Change_RNNPara_Fun(mPreData_df,20)
print(mPreDataRnn_x.shape,mPreDataRnn_y.shape)
print(mPreDataRnn_x[1])
#"""

(1479, 20, 8) (1479,)
[[-6.84695052e-01 -4.18526786e-01 -3.93133998e-01  3.71977681e-02
  -3.25303322e+00  5.17836594e-02  0.00000000e+00  2.81933257e-01]
 [-1.00061767e+00 -9.08561444e-01 -7.83861671e-01 -1.00061767e+00
  -2.20508020e-01  1.17356393e-01  0.00000000e+00  3.21185917e-01]
 [ 1.23541524e-01 -6.66239590e-01 -6.25390869e-01 -9.74605353e-01
  -1.55219938e-01  8.23610158e-02  4.87302677e-01 -6.17707618e-02]
 [ 1.62711864e-01  1.57858614e-01  1.73448966e-01  1.03422053e+00
  -1.03720084e+00  8.81355932e-02  9.83050847e-02 -6.44067797e-02]
 [-3.20213476e-01  1.35135135e-01  2.16393443e-01 -2.13645762e-01
   1.35687675e+00  1.93462308e-01  1.03402268e-01  1.63442295e-01]
 [-9.99310820e-01 -6.66666667e-01 -8.85750963e-01 -8.02816901e-01
   1.02079877e+00  6.89179876e-02  0.00000000e+00  3.23914542e-01]
 [-7.12098009e-01 -2.85714286e-01 -3.52112676e-01 -1.00306279e+00
   2.73472247e-01  3.82848392e-02  1.45482389e-01  5.62787136e-01]
 [ 9.97526793e-01 -8.01470588e-01 -2.62773723e-

In [57]:
#"""单支股票进行训练
#将Y转为one-hot向量
mPreDataRnnOne_y = mYChange_One_hot_Fun(mPreDataRnn_y)

#训练、测试集分离 
mXTrain_np = mPreDataRnn_x[:-800]
mYTrain = mPreDataRnnOne_y[:-800]

mXTest_np = mPreDataRnn_x[-800:-500]
mYTest_np = mPreDataRnnOne_y[-800:-500]

mXShape = mXTrain_np[0].shape
mYshape = mYTrain[0].shape
#"""

In [58]:
#搭建RNN模型
#--------------------1Rnn2MLP_tanh_softmax_10_5_5_4_SGD_0.001_
def mCreate_Rnn_Model_Fun(mRnnXDataShape,mRnnYshape):
    
    mRnnModel = Sequential()
    mRnnModel.add(SimpleRNN(units = 20,input_shape=(mRnnXDataShape[0],mRnnXDataShape[1]),activation='tanh'\
                            ,name='mFirst_Rnn_layer'))#,return_sequences='True'
    
    #mRnnModel.add(SimpleRNN(units = 10,activation='sigmoid',name='mSecond_Rnn_layer'))#sigmoid
    
    mRnnModel.add(Dense(units=10,activation='tanh',name='mFourth_MLP_layers'))
    
    mRnnModel.add(Dense(units=mRnnYshape[0],activation='softmax',name='mSecond_MLP_layers'))
    
    mRnnModel.summary()
    
    
    #优化器选择与设置
    #mOptimizer_obj = RMSprop(learning_rate=0.001)
    mOptimizer_obj =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    mRnnModel.compile(loss='categorical_crossentropy',optimizer=mOptimizer_obj,metrics=['accuracy'])
    
    return mRnnModel


In [59]:
#搭建RNN_LSTM模型
#--------------------1LSTM2MLP_tanh_softmax_10_5_5_4_SGD_0.001_
def mCreate_RnnLSTM_Model_Fun(mRnnXDataShape,mRnnYshape):
    
    mLSTMModel = Sequential()
    mLSTMModel.add(LSTM(units = 20,input_shape=(mRnnXDataShape[0],mRnnXDataShape[1]),activation='tanh'\
                            ,name='mFirst_Rnn_layer'))#,return_sequences='True'
    
    #mLSTMModel.add(LSTM(units = 10,activation='sigmoid',name='mSecond_Rnn_layer'))#sigmoid
    
    mLSTMModel.add(Dense(units=10,activation='tanh',name='mFourth_MLP_layers'))
    
    mLSTMModel.add(Dense(units=mRnnYshape[0],activation='softmax',name='mSecond_MLP_layers'))
    
    mLSTMModel.summary()
    
    
    #优化器选择与设置
    #mOptimizer_obj = RMSprop(learning_rate=0.001)
    mOptimizer_obj =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    mLSTMModel.compile(loss='categorical_crossentropy',optimizer=mOptimizer_obj,metrics=['accuracy'])
    
    return mLSTMModel


In [60]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten
#搭建CNN_Conv2D模型
#--------------------2Rnn2MLP_tanh_softmax_10_5_5_4_SGD_0.001_
def mCreate_SimpleCNN_Model_Fun(mRnnXDataShape,mRnnYshape):
    
    mCnnModel = Sequential()
    mCnnModel.add(Conv2D(32,(3,3),input_shape=(20,8,1),activation='tanh'))
    
    mCnnModel.add(MaxPool2D(pool_size=(2,2)))
    
    mCnnModel.add(Flatten())
   
    mCnnModel.add(Dense(units=10,activation='tanh',name='mFirst_MLP_layers'))
    
    mCnnModel.add(Dense(units=mRnnYshape[0],activation='softmax',name='mSecond_MLP_layers'))
    
    #mCnnModel.summary()
    
    
    #优化器选择与设置
    mOptimizer_obj =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    mCnnModel.compile(loss='categorical_crossentropy',optimizer=mOptimizer_obj,metrics=['accuracy'])
    
    return mCnnModel


In [61]:
#"""
mRnnModel_model = mCreate_Rnn_Model_Fun(mXShape,mYshape) 
    
#对分类为3 重点关注
mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}
    
early_stopping = EarlyStopping(monitor='accuracy', patience=20)
#,callbacks=[early_stopping]
mTrainResult_obj = mRnnModel_model.fit(mXTrain_np,mYTrain,class_weight=mClaseWeight_dic,epochs=1000,batch_size=150,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);
#"""

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mFirst_Rnn_layer (SimpleRNN) (None, 20)                580       
_________________________________________________________________
mFourth_MLP_layers (Dense)   (None, 10)                210       
_________________________________________________________________
mSecond_MLP_layers (Dense)   (None, 5)                 55        
Total params: 845
Trainable params: 845
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/1000
5/5 [==============================] - 6s 100ms/step - loss: 2.7170 - accuracy: 0.2933 - val_loss: 1.6595 - val_accuracy: 0.2533
Epoch 2/1000
5/5 [==============================] - 0s 8ms/step - loss: 2.6235 - accuracy: 0.2815 - val_l

5/5 [==============================] - 0s 6ms/step - loss: 1.8775 - accuracy: 0.4490 - val_loss: 1.5949 - val_accuracy: 0.2900
Epoch 52/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8346 - accuracy: 0.4594 - val_loss: 1.6001 - val_accuracy: 0.2900
Epoch 53/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8078 - accuracy: 0.4549 - val_loss: 1.6009 - val_accuracy: 0.2833
Epoch 54/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8332 - accuracy: 0.4534 - val_loss: 1.5980 - val_accuracy: 0.2833
Epoch 55/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8298 - accuracy: 0.4566 - val_loss: 1.6024 - val_accuracy: 0.2833
Epoch 56/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8030 - accuracy: 0.4705 - val_loss: 1.6007 - val_accuracy: 0.2833
Epoch 57/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8269 - accuracy: 0.4456 - val_loss: 1.6020 - val_accuracy: 0.2833
Epoch 58/1000
5/5 [========

5/5 [==============================] - 0s 6ms/step - loss: 1.4341 - accuracy: 0.5591 - val_loss: 1.8630 - val_accuracy: 0.3033
Epoch 110/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.3932 - accuracy: 0.5516 - val_loss: 1.9031 - val_accuracy: 0.3100
Epoch 111/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.3819 - accuracy: 0.5578 - val_loss: 1.8548 - val_accuracy: 0.3033
Epoch 112/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4198 - accuracy: 0.5687 - val_loss: 1.9137 - val_accuracy: 0.3100
Epoch 113/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.3849 - accuracy: 0.5699 - val_loss: 1.8778 - val_accuracy: 0.3133
Epoch 114/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4059 - accuracy: 0.5693 - val_loss: 1.9206 - val_accuracy: 0.3033
Epoch 115/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.3787 - accuracy: 0.5509 - val_loss: 1.8740 - val_accuracy: 0.3000
Epoch 116/1000
5/5 [=

Epoch 167/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2104 - accuracy: 0.6069 - val_loss: 2.2497 - val_accuracy: 0.2767
Epoch 168/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1973 - accuracy: 0.6378 - val_loss: 2.2872 - val_accuracy: 0.2667
Epoch 169/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1695 - accuracy: 0.6326 - val_loss: 2.2283 - val_accuracy: 0.2800
Epoch 170/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1501 - accuracy: 0.6667 - val_loss: 2.3017 - val_accuracy: 0.2700
Epoch 171/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1621 - accuracy: 0.6463 - val_loss: 2.2980 - val_accuracy: 0.2800
Epoch 172/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1185 - accuracy: 0.6710 - val_loss: 2.2963 - val_accuracy: 0.2767
Epoch 173/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1225 - accuracy: 0.6680 - val_loss: 2.2868 - val_accuracy: 0.2700
Epoch 

Epoch 225/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0378 - accuracy: 0.6794 - val_loss: 2.5810 - val_accuracy: 0.2700
Epoch 226/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0138 - accuracy: 0.6706 - val_loss: 2.6337 - val_accuracy: 0.2733
Epoch 227/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0219 - accuracy: 0.6728 - val_loss: 2.6194 - val_accuracy: 0.2700
Epoch 228/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0154 - accuracy: 0.6979 - val_loss: 2.6575 - val_accuracy: 0.2800
Epoch 229/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.9832 - accuracy: 0.6842 - val_loss: 2.5977 - val_accuracy: 0.2833
Epoch 230/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.9956 - accuracy: 0.6869 - val_loss: 2.6627 - val_accuracy: 0.2767
Epoch 231/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0330 - accuracy: 0.6756 - val_loss: 2.6441 - val_accuracy: 0.2767
Epoch 

In [62]:
#"""
mLSTMModel_model = mCreate_RnnLSTM_Model_Fun(mXShape,mYshape) 
    
#对分类为3 重点关注
mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}
    
early_stopping = EarlyStopping(monitor='accuracy', patience=20)
#,callbacks=[early_stopping]
mTrainResult_obj = mLSTMModel_model.fit(mXTrain_np,mYTrain,class_weight=mClaseWeight_dic,epochs=1000,batch_size=150,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);
#"""

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mFirst_Rnn_layer (LSTM)      (None, 20)                2320      
_________________________________________________________________
mFourth_MLP_layers (Dense)   (None, 10)                210       
_________________________________________________________________
mSecond_MLP_layers (Dense)   (None, 5)                 55        
Total params: 2,585
Trainable params: 2,585
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
5/5 [==============================] - 3s 111ms/step - loss: 2.5938 - accuracy: 0.2181 - val_loss: 1.6093 - val_accuracy: 0.2200
Epoch 2/1000
5/5 [==============================] - 0s 9ms/step - loss: 2.5346 - accuracy: 0.2685 - val_loss: 1.5956 - val_accuracy: 0.2200
Epoch 3/1000
5/5 [==============================] - 0s 8ms/step - loss: 2.4911 - accuracy: 0.2790 -

Epoch 54/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.9599 - accuracy: 0.3796 - val_loss: 1.5076 - val_accuracy: 0.2633
Epoch 55/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.9442 - accuracy: 0.4184 - val_loss: 1.5131 - val_accuracy: 0.2667
Epoch 56/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.9074 - accuracy: 0.3997 - val_loss: 1.4961 - val_accuracy: 0.2633
Epoch 57/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.9075 - accuracy: 0.4157 - val_loss: 1.4964 - val_accuracy: 0.2600
Epoch 58/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.8922 - accuracy: 0.4206 - val_loss: 1.5246 - val_accuracy: 0.2733
Epoch 59/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.8714 - accuracy: 0.4126 - val_loss: 1.4763 - val_accuracy: 0.2767
Epoch 60/1000
5/5 [==============================] - 0s 10ms/step - loss: 1.9063 - accuracy: 0.3881 - val_loss: 1.4994 - val_accuracy: 0.2600
Epoch 61/100

Epoch 112/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5193 - accuracy: 0.5325 - val_loss: 1.6112 - val_accuracy: 0.2533
Epoch 113/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5752 - accuracy: 0.4982 - val_loss: 1.5363 - val_accuracy: 0.2933
Epoch 114/1000
5/5 [==============================] - 0s 10ms/step - loss: 1.5314 - accuracy: 0.5212 - val_loss: 1.5355 - val_accuracy: 0.2967
Epoch 115/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5271 - accuracy: 0.5359 - val_loss: 1.5729 - val_accuracy: 0.2700
Epoch 116/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.4839 - accuracy: 0.5343 - val_loss: 1.5718 - val_accuracy: 0.2767
Epoch 117/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5359 - accuracy: 0.5058 - val_loss: 1.5907 - val_accuracy: 0.2800
Epoch 118/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5124 - accuracy: 0.5262 - val_loss: 1.6167 - val_accuracy: 0.2800
Epoch

5/5 [==============================] - 0s 8ms/step - loss: 1.2966 - accuracy: 0.5911 - val_loss: 1.9410 - val_accuracy: 0.2500
Epoch 170/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.2961 - accuracy: 0.6020 - val_loss: 2.0372 - val_accuracy: 0.2600
Epoch 171/1000
5/5 [==============================] - 0s 8ms/step - loss: 1.2370 - accuracy: 0.6141 - val_loss: 1.9957 - val_accuracy: 0.2400
Epoch 172/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.2416 - accuracy: 0.6156 - val_loss: 2.0264 - val_accuracy: 0.2533
Epoch 173/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.2099 - accuracy: 0.6182 - val_loss: 2.1144 - val_accuracy: 0.2467
Epoch 174/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.2559 - accuracy: 0.6012 - val_loss: 2.0647 - val_accuracy: 0.2567
Epoch 175/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.2042 - accuracy: 0.6288 - val_loss: 2.0239 - val_accuracy: 0.2467
Epoch 176/1000
5/5 [=

Epoch 227/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9977 - accuracy: 0.7021 - val_loss: 2.4684 - val_accuracy: 0.2500
Epoch 228/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9978 - accuracy: 0.7108 - val_loss: 2.7624 - val_accuracy: 0.2533
Epoch 229/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9790 - accuracy: 0.6934 - val_loss: 2.5736 - val_accuracy: 0.2467
Epoch 230/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9961 - accuracy: 0.7012 - val_loss: 2.4815 - val_accuracy: 0.2367
Epoch 231/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9441 - accuracy: 0.7175 - val_loss: 2.5386 - val_accuracy: 0.2400
Epoch 232/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9028 - accuracy: 0.7205 - val_loss: 2.6400 - val_accuracy: 0.2533
Epoch 233/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9327 - accuracy: 0.7202 - val_loss: 2.6616 - val_accuracy: 0.2600
Epoch 

5/5 [==============================] - 0s 9ms/step - loss: 0.8473 - accuracy: 0.7601 - val_loss: 3.1062 - val_accuracy: 0.2633
Epoch 285/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.7910 - accuracy: 0.7675 - val_loss: 3.2772 - val_accuracy: 0.2800
Epoch 286/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.8403 - accuracy: 0.7289 - val_loss: 3.1184 - val_accuracy: 0.2600
Epoch 287/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.7932 - accuracy: 0.7427 - val_loss: 2.9656 - val_accuracy: 0.2733
Epoch 288/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.8178 - accuracy: 0.7678 - val_loss: 3.3347 - val_accuracy: 0.2467
Epoch 289/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.7442 - accuracy: 0.7720 - val_loss: 3.0275 - val_accuracy: 0.2600
Epoch 290/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.7790 - accuracy: 0.7623 - val_loss: 3.2592 - val_accuracy: 0.2500
Epoch 291/1000
5/5 [

Epoch 342/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.6987 - accuracy: 0.7873 - val_loss: 3.7413 - val_accuracy: 0.2667
Epoch 343/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.6794 - accuracy: 0.7780 - val_loss: 3.7861 - val_accuracy: 0.2533
Epoch 344/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.6432 - accuracy: 0.8024 - val_loss: 3.5490 - val_accuracy: 0.2900
Epoch 345/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.6801 - accuracy: 0.8115 - val_loss: 3.8135 - val_accuracy: 0.2667
Epoch 346/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.6306 - accuracy: 0.8055 - val_loss: 3.6668 - val_accuracy: 0.2800
Epoch 347/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.6255 - accuracy: 0.8093 - val_loss: 3.6897 - val_accuracy: 0.2700
Epoch 348/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.6249 - accuracy: 0.8206 - val_loss: 3.8362 - val_accuracy: 0.2667
E

5/5 [==============================] - 0s 9ms/step - loss: 0.6892 - accuracy: 0.8042 - val_loss: 4.1476 - val_accuracy: 0.2733
Epoch 400/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.5976 - accuracy: 0.8218 - val_loss: 4.3994 - val_accuracy: 0.2567
Epoch 401/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.5850 - accuracy: 0.8176 - val_loss: 4.1048 - val_accuracy: 0.2667


In [63]:
mCnnTrainData_np = mXTrain_np.reshape(679,20,8,1)

mCnnModel_model = mCreate_SimpleCNN_Model_Fun(mCnnTrainData_np,mYshape) 
    
#对分类为3 重点关注
mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}
    
early_stopping = EarlyStopping(monitor='accuracy', patience=20)
#,callbacks=[early_stopping]
mTrainResult_obj = mCnnModel_model.fit(mCnnTrainData_np,mYTrain,epochs=1000,batch_size=150,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);

Epoch 1/1000
5/5 [==============================] - 1s 61ms/step - loss: 1.6677 - accuracy: 0.2540 - val_loss: 1.6288 - val_accuracy: 0.2633
Epoch 2/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5765 - accuracy: 0.3001 - val_loss: 1.5469 - val_accuracy: 0.2867
Epoch 3/1000
5/5 [==============================] - 0s 10ms/step - loss: 1.5067 - accuracy: 0.3504 - val_loss: 1.5461 - val_accuracy: 0.2967
Epoch 4/1000
5/5 [==============================] - 0s 9ms/step - loss: 1.5000 - accuracy: 0.3228 - val_loss: 1.5448 - val_accuracy: 0.2867
Epoch 5/1000
5/5 [==============================] - 0s 8ms/step - loss: 1.4737 - accuracy: 0.3581 - val_loss: 1.5568 - val_accuracy: 0.2667
Epoch 6/1000
5/5 [==============================] - 0s 8ms/step - loss: 1.4586 - accuracy: 0.3577 - val_loss: 1.5363 - val_accuracy: 0.2867
Epoch 7/1000
5/5 [==============================] - 0s 8ms/step - loss: 1.4311 - accuracy: 0.3909 - val_loss: 1.5395 - val_accuracy: 0.2833
Epoch 8/1000
5/5 [

5/5 [==============================] - 0s 12ms/step - loss: 0.9561 - accuracy: 0.6995 - val_loss: 1.7035 - val_accuracy: 0.2500
Epoch 60/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.9550 - accuracy: 0.6579 - val_loss: 1.6877 - val_accuracy: 0.2600
Epoch 61/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.9472 - accuracy: 0.6785 - val_loss: 1.6989 - val_accuracy: 0.2667
Epoch 62/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.9506 - accuracy: 0.6853 - val_loss: 1.7047 - val_accuracy: 0.2400
Epoch 63/1000
5/5 [==============================] - 0s 7ms/step - loss: 0.9372 - accuracy: 0.6876 - val_loss: 1.7019 - val_accuracy: 0.2600
Epoch 64/1000
5/5 [==============================] - 0s 7ms/step - loss: 0.9172 - accuracy: 0.7066 - val_loss: 1.7118 - val_accuracy: 0.2600
Epoch 65/1000
5/5 [==============================] - 0s 7ms/step - loss: 0.9086 - accuracy: 0.7009 - val_loss: 1.7153 - val_accuracy: 0.2467
Epoch 66/1000
5/5 [======

5/5 [==============================] - 0s 8ms/step - loss: 0.5553 - accuracy: 0.8797 - val_loss: 1.9198 - val_accuracy: 0.2767
Epoch 118/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.5339 - accuracy: 0.8764 - val_loss: 1.9289 - val_accuracy: 0.2633
Epoch 119/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.5510 - accuracy: 0.8712 - val_loss: 1.9309 - val_accuracy: 0.2600
Epoch 120/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.5409 - accuracy: 0.8815 - val_loss: 1.9334 - val_accuracy: 0.2800
Epoch 121/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.5258 - accuracy: 0.8781 - val_loss: 1.9471 - val_accuracy: 0.2667
Epoch 122/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.5316 - accuracy: 0.8873 - val_loss: 1.9485 - val_accuracy: 0.2700
Epoch 123/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.5176 - accuracy: 0.9027 - val_loss: 1.9523 - val_accuracy: 0.2800
Epoch 124/1000
5/5 [

Epoch 175/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2741 - accuracy: 0.9748 - val_loss: 2.2184 - val_accuracy: 0.2633
Epoch 176/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2781 - accuracy: 0.9778 - val_loss: 2.2198 - val_accuracy: 0.2667
Epoch 177/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2609 - accuracy: 0.9828 - val_loss: 2.2206 - val_accuracy: 0.2633
Epoch 178/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2634 - accuracy: 0.9765 - val_loss: 2.2300 - val_accuracy: 0.2567
Epoch 179/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2634 - accuracy: 0.9763 - val_loss: 2.2358 - val_accuracy: 0.2567
Epoch 180/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2591 - accuracy: 0.9760 - val_loss: 2.2379 - val_accuracy: 0.2500
Epoch 181/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.2524 - accuracy: 0.9775 - val_loss: 2.2468 - val_accuracy: 0.2600
Epoch 

Epoch 233/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.1370 - accuracy: 0.9944 - val_loss: 2.5047 - val_accuracy: 0.2567
Epoch 234/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.1316 - accuracy: 0.9985 - val_loss: 2.5231 - val_accuracy: 0.2567
Epoch 235/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.1312 - accuracy: 0.9975 - val_loss: 2.5281 - val_accuracy: 0.2633
Epoch 236/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.1280 - accuracy: 0.9978 - val_loss: 2.5278 - val_accuracy: 0.2567
Epoch 237/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.1292 - accuracy: 0.9955 - val_loss: 2.5439 - val_accuracy: 0.2533
Epoch 238/1000
5/5 [==============================] - 0s 8ms/step - loss: 0.1281 - accuracy: 0.9972 - val_loss: 2.5381 - val_accuracy: 0.2500
Epoch 239/1000
5/5 [==============================] - 0s 9ms/step - loss: 0.1252 - accuracy: 0.9984 - val_loss: 2.5559 - val_accuracy: 0.2567
Epoch 

In [37]:

"""
mRnnModel_model = mCreate_Rnn_Model_Fun(mXShape,mYshape) 
    
#对分类为3 重点关注
mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}
    
early_stopping = EarlyStopping(monitor='accuracy', patience=20)
#,callbacks=[early_stopping]
mTrainResult_obj = mRnnModel_model.fit(mXTrain_np,mYTrain,class_weight=mClaseWeight_dic,epochs=1000,batch_size=150,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);
"""

D:\Anaconda\envs\csdn_ml_env\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mFirst_Rnn_layer (SimpleRNN) (None, 20)                580       
_________________________________________________________________
mFourth_MLP_layers (Dense)   (None, 10)                210       
_________________________________________________________________
mSecond_MLP_layers (Dense)   (None, 5)                 55        
Total params: 845
Trainable params: 845
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
5/5 [==============================] - 2s 107ms/step - loss: 2.7648 - accuracy: 0.2040 - val_loss: 1.6989 - val_accuracy: 0.2100
Epoch 2/1000
5/5 [==============================] - 0s 7ms/step - loss: 2.7232 - accuracy: 0.2001 - val_loss: 1.6747 - val_accuracy: 0.2100
Epoch 3/1000
5/5 [==============================] - 0s 7ms/step - loss: 2.6921 - accuracy: 0.1992 - val_

Epoch 54/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8477 - accuracy: 0.4492 - val_loss: 1.5754 - val_accuracy: 0.2900
Epoch 55/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8205 - accuracy: 0.4464 - val_loss: 1.5210 - val_accuracy: 0.2833
Epoch 56/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.8139 - accuracy: 0.4503 - val_loss: 1.5706 - val_accuracy: 0.2900
Epoch 57/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.7631 - accuracy: 0.4648 - val_loss: 1.5256 - val_accuracy: 0.2800
Epoch 58/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.7865 - accuracy: 0.4641 - val_loss: 1.5351 - val_accuracy: 0.2833
Epoch 59/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.7773 - accuracy: 0.4498 - val_loss: 1.5584 - val_accuracy: 0.2833
Epoch 60/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.7889 - accuracy: 0.4419 - val_loss: 1.5338 - val_accuracy: 0.2833
Epoch 61/1000

5/5 [==============================] - 0s 6ms/step - loss: 1.4310 - accuracy: 0.5764 - val_loss: 1.7373 - val_accuracy: 0.2867
Epoch 113/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4692 - accuracy: 0.5414 - val_loss: 1.7062 - val_accuracy: 0.2867
Epoch 114/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4256 - accuracy: 0.5910 - val_loss: 1.7142 - val_accuracy: 0.2967
Epoch 115/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4407 - accuracy: 0.5892 - val_loss: 1.7416 - val_accuracy: 0.2700
Epoch 116/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4815 - accuracy: 0.5707 - val_loss: 1.7403 - val_accuracy: 0.2700
Epoch 117/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4086 - accuracy: 0.5960 - val_loss: 1.7642 - val_accuracy: 0.2867
Epoch 118/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.4337 - accuracy: 0.5490 - val_loss: 1.7426 - val_accuracy: 0.2867
Epoch 119/1000
5/5 [=

Epoch 170/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2574 - accuracy: 0.6487 - val_loss: 1.9647 - val_accuracy: 0.2733
Epoch 171/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2804 - accuracy: 0.6027 - val_loss: 1.9509 - val_accuracy: 0.2900
Epoch 172/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2499 - accuracy: 0.6228 - val_loss: 1.9448 - val_accuracy: 0.2900
Epoch 173/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2028 - accuracy: 0.6533 - val_loss: 1.9794 - val_accuracy: 0.3067
Epoch 174/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2499 - accuracy: 0.6321 - val_loss: 1.9639 - val_accuracy: 0.3100
Epoch 175/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.1839 - accuracy: 0.6506 - val_loss: 1.9586 - val_accuracy: 0.3000
Epoch 176/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.2564 - accuracy: 0.6483 - val_loss: 1.9779 - val_accuracy: 0.3000
Epoch 

Epoch 228/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0619 - accuracy: 0.6749 - val_loss: 2.1038 - val_accuracy: 0.2900
Epoch 229/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0555 - accuracy: 0.6461 - val_loss: 2.0797 - val_accuracy: 0.2933
Epoch 230/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0521 - accuracy: 0.6793 - val_loss: 2.1021 - val_accuracy: 0.2867
Epoch 231/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0483 - accuracy: 0.6531 - val_loss: 2.0661 - val_accuracy: 0.3000
Epoch 232/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0546 - accuracy: 0.6736 - val_loss: 2.0927 - val_accuracy: 0.2933
Epoch 233/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0210 - accuracy: 0.6782 - val_loss: 2.1228 - val_accuracy: 0.2867
Epoch 234/1000
5/5 [==============================] - 0s 6ms/step - loss: 1.0492 - accuracy: 0.6648 - val_loss: 2.0759 - val_accuracy: 0.3000
Epoch 

Epoch 286/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.8989 - accuracy: 0.7075 - val_loss: 2.2946 - val_accuracy: 0.2967
Epoch 287/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.9139 - accuracy: 0.7226 - val_loss: 2.3471 - val_accuracy: 0.2933
Epoch 288/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.9132 - accuracy: 0.7084 - val_loss: 2.3142 - val_accuracy: 0.2767
Epoch 289/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.9852 - accuracy: 0.7037 - val_loss: 2.3011 - val_accuracy: 0.2800
Epoch 290/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.9047 - accuracy: 0.7242 - val_loss: 2.2742 - val_accuracy: 0.2833
Epoch 291/1000
5/5 [==============================] - 0s 6ms/step - loss: 0.8908 - accuracy: 0.7398 - val_loss: 2.3613 - val_accuracy: 0.2867
Epoch 292/1000
5/5 [==============================] - 0s 7ms/step - loss: 0.9329 - accuracy: 0.6959 - val_loss: 2.2968 - val_accuracy: 0.2767
Epoch 

In [64]:
mYPredict_np = mRnnModel_model.predict(mXTrain_np)
print(mYPredict_np)

[[8.4843856e-01 1.2505791e-01 1.9791368e-02 5.9282999e-03 7.8381237e-04]
 [3.6709673e-05 3.8274360e-04 1.0186367e-03 1.7732799e-02 9.8082918e-01]
 [1.2018472e-05 1.4831572e-04 2.2557913e-03 2.0954274e-02 9.7662967e-01]
 ...
 [1.2663649e-03 4.5100991e-02 2.0106266e-01 7.4905998e-01 3.5100300e-03]
 [1.8339857e-02 1.7816120e-01 3.7617686e-01 4.2689136e-01 4.3075535e-04]
 [7.4324876e-02 3.7538862e-01 4.1377196e-01 1.3622546e-01 2.8915770e-04]]


In [65]:
print(mYPredict_np.shape,mXTrain_np.shape)

(679, 5) (679, 20, 8)


In [66]:
#封装数据预处理
def mGet_RNN_FinalPreDate_Fun(mStockPath_str):
    #数据读取
    mOrgData_df = mLoad_Data_Fun(mStockPath_str)
    #数据预处理，主要是数据的归一化
    mPreData_df = mPreDeal_Total_Fun(mOrgData_df)
    #数据预处理，将数据转为RNN神经网络的输入
    mPreDataRnn_x,mPreDataRnn_y = mPre_Change_RNNPara_Fun(mPreData_df,20)

    #将Y转为one-hot向量
    mPreDataRnnOne_y = mYChange_One_hot_Fun(mPreDataRnn_y)

    
    #训练、测试集分离
    #测试集所占比率
    mTestRate_D = 0.2
    mTestLocation = int(len(mPreDataRnn_x) * mTestRate_D)
    
    print(mTestLocation)
    mXTrain_np = mPreDataRnn_x[:-mTestLocation]
    mYTrain_np = mPreDataRnnOne_y[:-mTestLocation]

    mXTest_np = mPreDataRnn_x[-mTestLocation:]
    mYTest_np = mPreDataRnnOne_y[-mTestLocation:]
    
    return mXTrain_np,mYTrain_np,mXTest_np,mYTest_np


In [67]:
#一支股票的训练'D:\\mMLP\\StockData\\mTrainStock\\600056.csv'
def mRNN_Train_OneStock_Fun(mStockDataPath_str):
    #数据读取
    mXTrain_np,mYTrain_np,mXTest_np,mYTest_np = mGet_RNN_FinalPreDate_Fun(mStockDataPath_str)

    mXShape = mXTrain_np[0].shape
    mYshape = mYTrain_np[0].shape


    mRnnModel_model = mCreate_Rnn_Model_Fun(mXShape,mYshape) 

    #对分类为3 重点关注
    mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}

    early_stopping = EarlyStopping(monitor='accuracy', patience=10)
    #,callbacks=[early_stopping],class_weight=mClaseWeight_dic,
    mTrainResult_obj = mRnnModel_model.fit(mXTrain_np,mYTrain_np,epochs=1000,batch_size=100,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);
    mAccuray = mTrainResult_obj.history['val_accuracy']
    if mAccuray[-1] > 0.5:
        mSymbol_str = mStockDataPath_str[-10:-4]
        print(mSymbol_str,' accuracy ',mAccuray[-1])
        mNeedWriteContent_str = mSymbol_str + ' accuracy ' + str(mAccuray[-1]) + '\n'
        mRecordFile_str = 'D:\\mMLP\\StockData\\mResult\\mSatisfy.txt'
        
        with open(mRecordFile_str,'a+') as mSave_obj:
            mSave_obj.write(mNeedWriteContent_str)

In [68]:
#对所有股票进行模型训练，对满足要求的模型进行模型存储。
def mRnn_Train_AllStock_Fun():
    mFilePath_str = 'E:\\mMachineLerning\\Own\\RNN\\mTrain_StockData\\'
    mSymbolList_np = os.listdir(mFilePath_str)
    
    i = 0
    while(i <len(mSymbolList_np)):
        mDataPath_str = 'E:\\mMachineLerning\\Own\\RNN\\mTrain_StockData\\' + mSymbolList_np[i][:6] + '.csv'
        print(i,'   ',mDataPath_str)
        mRNN_Train_OneStock_Fun(mDataPath_str)
        if i > 100:
            return
        i += 1
    #mRNN_Train_OneStock_Fun('D:\\mMLP\\StockData\\mTrainStock\\600820.csv')

In [69]:
mRnn_Train_AllStock_Fun()

0     E:\mMachineLerning\Own\RNN\mTrain_StockData\000012.csv
295
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mFirst_Rnn_layer (SimpleRNN) (None, 20)                580       
_________________________________________________________________
mFourth_MLP_layers (Dense)   (None, 10)                210       
_________________________________________________________________
mSecond_MLP_layers (Dense)   (None, 5)                 55        
Total params: 845
Trainable params: 845
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
12/12 [==============================] - 2s 42ms/step - loss: 1.6771 - accuracy: 0.2445 - val_loss: 1.6428 - val_accuracy: 0.2746
Epoch 2/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.6103 - accuracy: 0.2625 - val_loss: 1.6169 - val_accuracy: 0.2847
Epoch 3/1000
12/12 [===============

12/12 [==============================] - 0s 3ms/step - loss: 1.1843 - accuracy: 0.4676 - val_loss: 1.2529 - val_accuracy: 0.4949
Epoch 53/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.2027 - accuracy: 0.4639 - val_loss: 1.2480 - val_accuracy: 0.5017
Epoch 54/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.1767 - accuracy: 0.4899 - val_loss: 1.2487 - val_accuracy: 0.4915
Epoch 55/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.1694 - accuracy: 0.4951 - val_loss: 1.2540 - val_accuracy: 0.4915
Epoch 56/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.2121 - accuracy: 0.4786 - val_loss: 1.2392 - val_accuracy: 0.5153
Epoch 57/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.1647 - accuracy: 0.5091 - val_loss: 1.2551 - val_accuracy: 0.4983
Epoch 58/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.1616 - accuracy: 0.4971 - val_loss: 1.2422 - val_accuracy: 0.5119
Epoch 59/1000

12/12 [==============================] - 0s 3ms/step - loss: 1.0095 - accuracy: 0.5714 - val_loss: 1.3034 - val_accuracy: 0.4949
Epoch 110/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.0197 - accuracy: 0.5793 - val_loss: 1.3039 - val_accuracy: 0.4915
Epoch 111/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.0197 - accuracy: 0.5529 - val_loss: 1.3172 - val_accuracy: 0.4881
Epoch 112/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.0017 - accuracy: 0.5659 - val_loss: 1.3185 - val_accuracy: 0.4949
Epoch 113/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.0021 - accuracy: 0.5775 - val_loss: 1.3215 - val_accuracy: 0.4915
Epoch 114/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.9854 - accuracy: 0.5803 - val_loss: 1.3136 - val_accuracy: 0.4949
Epoch 115/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.0181 - accuracy: 0.5667 - val_loss: 1.3289 - val_accuracy: 0.4881
Epoch 1

Epoch 166/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8157 - accuracy: 0.6717 - val_loss: 1.7444 - val_accuracy: 0.4305
Epoch 167/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8229 - accuracy: 0.6525 - val_loss: 1.7544 - val_accuracy: 0.4271
Epoch 168/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8070 - accuracy: 0.6708 - val_loss: 1.7757 - val_accuracy: 0.4102
Epoch 169/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8103 - accuracy: 0.6612 - val_loss: 1.7858 - val_accuracy: 0.4136
Epoch 170/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8006 - accuracy: 0.6630 - val_loss: 1.8028 - val_accuracy: 0.4305
Epoch 171/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8000 - accuracy: 0.6880 - val_loss: 1.7834 - val_accuracy: 0.4305
Epoch 172/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7984 - accuracy: 0.6893 - val_loss: 1.8026 - val_accuracy:

D:\Anaconda\envs\csdn_ml_env\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
D:\Anaconda\envs\csdn_ml_env\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


...

12/12 [==============================] - 0s 5ms/step - loss: 1.3420 - accuracy: 0.4262 - val_loss: 1.2625 - val_accuracy: 0.5288
Epoch 28/1000
12/12 [==============================] - 0s 5ms/step - loss: 1.3218 - accuracy: 0.4399 - val_loss: 1.2568 - val_accuracy: 0.5220
Epoch 29/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.3298 - accuracy: 0.4130 - val_loss: 1.2555 - val_accuracy: 0.5220
Epoch 30/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.3111 - accuracy: 0.4361 - val_loss: 1.2512 - val_accuracy: 0.5153
Epoch 31/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.2925 - accuracy: 0.4587 - val_loss: 1.2421 - val_accuracy: 0.5186
Epoch 32/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.2936 - accuracy: 0.4440 - val_loss: 1.2374 - val_accuracy: 0.5153
Epoch 33/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.2911 - accuracy: 0.4501 - val_loss: 1.2361 - val_accuracy: 0.4949
Epoch 34/1000

12/12 [==============================] - 0s 4ms/step - loss: 1.0281 - accuracy: 0.5646 - val_loss: 1.3952 - val_accuracy: 0.4949
Epoch 85/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.0301 - accuracy: 0.5667 - val_loss: 1.4109 - val_accuracy: 0.4780
Epoch 86/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.0181 - accuracy: 0.5682 - val_loss: 1.4288 - val_accuracy: 0.4780
Epoch 87/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.0287 - accuracy: 0.5604 - val_loss: 1.4196 - val_accuracy: 0.4847
Epoch 88/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.9984 - accuracy: 0.5829 - val_loss: 1.4442 - val_accuracy: 0.4610
Epoch 89/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.0098 - accuracy: 0.5766 - val_loss: 1.4511 - val_accuracy: 0.4712
Epoch 90/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.0062 - accuracy: 0.5831 - val_loss: 1.4464 - val_accuracy: 0.4847
Epoch 91/1000

12/12 [==============================] - 0s 4ms/step - loss: 0.8338 - accuracy: 0.6710 - val_loss: 1.8470 - val_accuracy: 0.4475
Epoch 142/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8218 - accuracy: 0.6816 - val_loss: 1.8405 - val_accuracy: 0.4441
Epoch 143/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8515 - accuracy: 0.6560 - val_loss: 1.8631 - val_accuracy: 0.4475
Epoch 144/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8364 - accuracy: 0.6688 - val_loss: 1.8702 - val_accuracy: 0.4441
Epoch 145/1000
12/12 [==============================] - ETA: 0s - loss: 0.9014 - accuracy: 0.60 - 0s 3ms/step - loss: 0.8562 - accuracy: 0.6421 - val_loss: 1.8756 - val_accuracy: 0.4441
Epoch 146/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8218 - accuracy: 0.6667 - val_loss: 1.8869 - val_accuracy: 0.4475
Epoch 147/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8359 - accuracy: 0.6458 - val_l

Epoch 198/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.7183 - accuracy: 0.6887 - val_loss: 2.2923 - val_accuracy: 0.4271
Epoch 199/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7042 - accuracy: 0.6969 - val_loss: 2.3079 - val_accuracy: 0.4203
Epoch 200/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7114 - accuracy: 0.7204 - val_loss: 2.3200 - val_accuracy: 0.4237
Epoch 201/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7179 - accuracy: 0.7203 - val_loss: 2.3524 - val_accuracy: 0.4136
Epoch 202/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7369 - accuracy: 0.6811 - val_loss: 2.3376 - val_accuracy: 0.4203
Epoch 203/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7156 - accuracy: 0.6985 - val_loss: 2.3408 - val_accuracy: 0.4237
Epoch 204/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.7252 - accuracy: 0.6926 - val_loss: 2.3552 - val_accuracy:

Epoch 255/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.6418 - accuracy: 0.7320 - val_loss: 2.7386 - val_accuracy: 0.3966
Epoch 256/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.6289 - accuracy: 0.7377 - val_loss: 2.7013 - val_accuracy: 0.4136
Epoch 257/1000
12/12 [==============================] - 0s 5ms/step - loss: 0.6321 - accuracy: 0.7527 - val_loss: 2.7609 - val_accuracy: 0.4034
Epoch 258/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.6176 - accuracy: 0.7564 - val_loss: 2.7239 - val_accuracy: 0.4068
Epoch 259/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.5921 - accuracy: 0.7663 - val_loss: 2.7940 - val_accuracy: 0.4102
Epoch 260/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.5848 - accuracy: 0.7684 - val_loss: 2.7407 - val_accuracy: 0.4068
Epoch 261/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.6163 - accuracy: 0.7538 - val_loss: 2.7729 - val_accuracy: